# Izračun razlike višine in azimuta nebesnega telesa (SRP - Sight Reduction Procedure)

Izračun parametrov položajnice po metodi *Mark De Saint Hailare*. Ročno poteka izračun z uporabo <a href="https://unilj-my.sharepoint.com/:b:/g/personal/aleksander_grm_fpp_uni-lj_si/EQhyqtLl3lNKsl2Jo21S4f8Bidd2DuJ_hLI1jO1bR5Cg8w">Navtičnega almanaha</a> (*Gha*, *Dec* in *Sha* pomladišča in nebesnega telesa) in meritvijo višine $H$ nebesnega telesa.

Poleg same meritve moramo določiti še čas meritve in položaj meritve. Položaje vnašamo v obliki, kot je v navtiki dogovor 

$\varphi = \#\#^\circ \, \#\#.\#' \, \mathrm{N/S}$, $\lambda = \#\#\#^\circ \, \#\#.\#' \, \mathrm{E/W}$, 

kjer imamo stopinje kot celo število in minute kot število na 1 decimalno mesto natančno.

Format vnosa položaja:
- LAT-$\varphi$ in LONG-$\lambda$ vnašamo v formatu [stopinje, minute, stran neba]

Primer vnosa položaja:
- $\varphi$: [12, 34.5, 'N']
- $\lambda$: [117, 12.6, 'W']
- stran neba mora biti v **enojnih navednicah**, kjer so dovoljeni znaki 'N', 'S', 'E', 'W'

Čas meritve zapišemo v obliki
- datum: *date*=[day,month,year]
- ura: *time*=[hour,minutes, seconds]

Format vpisa višine je enak kakor za položaj in zapišemo v obliki
- višina: $h=\#\#^\circ \#\#.\#'$=[deg,min] (primer: [34, 17.6])

<hr>

**Postopek izračuna**

Postopek določanja parametrov položajnice poteka po naslednjem vrstnem redu:

1. Najprej je potrebno izbrati domnevni položaj (apparent/assumed position)
   $$P_\mathrm{AP} = (\varphi_\mathrm{AP},\lambda_\mathrm{AP}),$$
2. Nato izračunamo višino merjenega nebesnega telesa (height calculated $H_c$) s pomočjo formule
   $$ \sin H_c = \sin \varphi_\mathrm{AP} \: \sin \delta + \cos \varphi_\mathrm{AP} \: \cos \delta \: \cos \mathrm{Lha},$$
3. Potem izračunamo azimut nebesnega telesa iz našega domnevnega položaja
   $$\cos \omega = \frac{\sin \delta - \sin H_c \: \sin \varphi_\mathrm{AP}}{\cos H_c \: \cos \varphi_\mathrm{AP}}$$ in ga nato pretvorimo v navtični azimut, ki ga potrebujemo za risanje položajne linije
   $$\omega_\mathrm{N} = \begin{cases} \omega &; \mathrm{če~je~} 180^\circ < \mathrm{Lha} \leq 360^\circ \\ 360^\circ - \omega &; \mathrm{če~je~} ~~~0^\circ < \mathrm{Lha}  \leq 180^\circ \end{cases}$$
  Tako smo izračunani azimut $\omega$ pretvorili v navtični azimut $\omega_\mathrm{N}$ s pomočjo podatka o mestnem časovnem kotu $\mathrm{Lha}$.
4. S pomočjo izmerjene in popravljene višine $H_\mathrm{o}$ (height observed) lahko določimo razliko v višinah, ki ji v angleščini pravimo **intercept** $$\Delta H [\mathrm{Nm}]= 60 \: (H_\mathrm{o}[\mathrm{deg}] - H_\mathrm{c}[\mathrm{deg}]).$$ Razlika se določi v Nm, ker potrebujemo razdaljo za vris položajnice na karto.

<hr>

## Nalaganje ustreznih modulov in program za izračun 


In [ ]:
import os, sys

# add custom modules and astro data path 
pp = '../nav_tools/'
sys.path.append(pp)

In [ ]:
import math as mat
import numpy as np
import matplotlib.pyplot as mpl
mpl.rcParams['text.usetex'] = True
mpl.rcParams.update({'font.size': 7})

import celestialdata as cdata
import navigationalstars as ns
import navtools as nt

In [ ]:
def getHeight(fi, dec, lha):

    fi_rad = nt.deg2rad(fi)
    dec_rad = nt.deg2rad(dec)
    lha_rad = nt.deg2rad(lha)

    sin_hc = mat.sin(fi_rad)*mat.sin(dec_rad) + mat.cos(fi_rad)*mat.cos(dec_rad)*mat.cos(lha_rad)

    return mat.asin(sin_hc) 

In [ ]:
def getAzimuth(fi, dec, lha, hc):

    fi_rad = nt.deg2rad(fi)
    dec_rad = nt.deg2rad(dec)
    lha_rad = nt.deg2rad(lha)

    cos_wc = (mat.sin(dec_rad) - mat.sin(hc)*mat.sin(fi_rad))/(mat.cos(hc)*mat.cos(fi_rad))

    wc = mat.acos(cos_wc)

    if lha > mat.pi:
        wn = wc
    else:
        wn = 2*mat.pi - wc

    return wn

In [ ]:
# poišče vsa navigacijska nebesna telesa nad obzorjem
def findAllVisibleCelestialBodies(t,pos,ns_db):
    
    star_ids = ns_db.keys()
    h0 = 0
    h1 = 90
    
    vs = []
    for s_id in star_ids:
        sdd = cd.get_star_data(s_id,t)
        saz = cd.get_star_altaz(s_id,t,pos)
        if saz['alt'] > h0 and saz['alt'] < h1:
            s_name = ns_db[s_id][0]
            vs.append({'name':s_name, 'dec':sdd['dec'], 'sha':sdd['sha'], 'alt':saz['alt'], 'az':saz['az']})
    
    return vs

In [ ]:
# poišči podatke o merjenem nebesnem telesu
def findCelestialBodyData(ns_db,pos,t,name):
 
    gha_a = cd.get_aries_gha(t)
    lha_a = gha_a + pos[1]

    cbs = findAllVisibleCelestialBodies(t,pos,ns_db)
    dec = 0
    sha = 0
    alt = 0
    az = 0
    found = False

    for cb in cbs:
        if cb['name'].upper() == name.upper():
            dec = cb['dec']
            sha = cb['sha']
            alt = cb['alt']
            az = cb['az']
            found = True

    data = {'gha_a':gha_a, 'lha_a':lha_a, 'dec':dec, 'sha':sha, 'alt':alt, 'az':az}

    return [found,data,cbs]   

In [ ]:
#
def getHeightAndAzimuth(ns_db,pos,date,time,name):

    t = [date[0],date[1],date[2],time[0],time[1],time[2]]

    [cf, data,cbs] = findCelestialBodyData(ns_db,pos,t,name)

    if cf:
        lha = data['lha_a'] + data['sha']
        hc = nt.rad2deg(getHeight(pos[0], data['dec'], lha))
        wn = nt.rad2deg(getAzimuth(pos[0], data['dec'], hc, lha))

        return [cf,hc,wn,cbs]
    else:
        return [cf,0,0,cbs] 

<hr>

## Vnos vhodnih podatkov in izračun


In [ ]:
# Load astro database
cd = cdata.CelestialData(pp)
ns_db = cd.get_nav_stars_db()

In [ ]:
# *** Inputs ***
date = [2022,2,12] # [yyyy, mm, dd]
time = [19,0,0]    # [HH:MM:SS] in UTC

# Apparent position
fi = [46,0,'N']; 
la = [13,30,'E']

# Observer height in [meters]
h = 20.0

# Observed Celestial Body 
cb_name = 'betelgeuse' # Celestial Body name
ho = [51, 11.28]       # Celestial Body observed height [deg, mi.dec] (corrected!!)


# -------------------
# --- calculation ---
# -------------------

pos = [nt.nav2dd(fi), nt.nav2dd(la), h]
ho_dd = ho[0]+ho[1]/60

[cf, hc, wn, cbs] = getHeightAndAzimuth(ns_db,pos,date,time,cb_name)

if cf:
    dh = 60 * (ho_dd - hc)
    print('Results of calculation:')
    print()
    print('  -> body: {:s}'.format(cb_name.upper()))
    print('  -> height calculated: {:s}'.format(nt.prettyPrintAlt(hc)))
    print('  -> height measured:   {:s}'.format(nt.prettyPrintAlt(ho_dd)))
    
    print()
    print('  -> azimuth:   {:s}'.format(nt.prettyPrintAz(wn)))
    print('  -> intercept: {:6.2f} Nm'.format(dh))
else:
    print('  ERROR: Celestial body name {:s} NOT found in database'.format(cb_name))
    print('         Visible bodies are:')
    for cb in cbs:
        print('           -> {:12s}'.format(cb['name']))
        